In [33]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [38]:
%sql sqlite:///danny.db

In [44]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///danny.db
Done.


name
sales
menu
members


### Case Study Questions

1. What is the total amount each customer spent at the restaurant?

In [57]:
%%sql
SELECT customer_id,
       sum(price) AS Amount
  FROM menu m
       JOIN
       sales s ON m.product_id = s.product_id
 GROUP BY customer_id;

 * sqlite:///danny.db
Done.


customer_id,Amount
A,76
B,74
C,36


2. How many days has each customer visited the restaurant?

In [58]:
%%sql
SELECT customer_id,
       count(DISTINCT (order_date) ) AS visited
  FROM sales
 GROUP BY customer_id;

 * sqlite:///danny.db
Done.


customer_id,visited
A,4
B,6
C,2


3. What was the first item from the menu purchased by each customer?

In [63]:
%%sql
SELECT customer_id,
       first_value(m.product_name) OVER (PARTITION BY customer_id ORDER BY order_date) AS first_item_purchased
  FROM sales s
       JOIN
       menu m ON s.product_id = m.product_id
 GROUP BY customer_id;


 * sqlite:///danny.db
Done.


customer_id,first_item_purchased
A,sushi
B,curry
C,ramen


4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [67]:
%%sql
SELECT m.product_name,
       count(*) AS most_purchased_item
  FROM sales s
       JOIN
       menu m ON s.product_id = m.product_id
 GROUP BY m.product_name
 ORDER BY most_purchased_item DESC
 LIMIT 1;


 * sqlite:///danny.db
Done.


product_name,most_purchased_item
ramen,8


5. Which item was the most popular for each customer?

In [68]:
%%sql
WITH cte AS (
    SELECT s.customer_id,
           m.product_name,
           COUNT( * ) AS purchase_count,
           row_number() OVER (PARTITION BY s.customer_id ORDER BY COUNT( * ) DESC) AS rank
      FROM sales s
           JOIN
           menu m ON s.product_id = m.product_id
     GROUP BY s.customer_id,
              m.product_name
)
SELECT customer_id,
       product_name, 
       purchase_count
  FROM cte
 WHERE rank = 1;


 * sqlite:///danny.db
Done.


customer_id,product_name,purchase_count
A,ramen,3
B,sushi,2
C,ramen,3
